# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [31]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
# Load csv
df = pd.read_csv('output_data/city_weather_data.csv')

# View snapshot of dataset
df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country
0,0,canico,32.6333,-16.8500,72.23,43,20,16.11,1650148882,PT
1,1,barrow,71.2906,-156.7887,6.82,72,0,21.85,1650148686,US
2,2,rikitea,-23.1203,-134.9692,78.49,70,7,13.67,1650148883,PF
3,3,taltal,-25.4000,-70.4833,62.49,77,22,5.86,1650148883,CL
4,4,cape town,-33.9258,18.4232,59.92,70,0,15.01,1650148715,ZA


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [66]:
# Bring in location data pairs, humidity data, define map and layer, and add layer to map
locations = df[["Lat", "Lng"]]
humidity = df["Humidity"]

# Plot heatmap
fig = gmaps.figure()

# Create and add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100,
                                 point_radius=3)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [22]:
# Set weather preferences
narrow_df = df.loc[(df['Max Temp'] > 65) & (df['Max Temp'] < 80) &
                  (df['Wind Speed'] < 10) & (df['Cloudiness'] == 0), :]

narrow_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country
11,11,mar del plata,-38.0023,-57.5575,65.08,89,0,3.00,1650148597,AR
51,51,buraydah,26.3260,43.9750,75.65,19,0,9.33,1650148901,SA
194,194,taoudenni,22.6783,-3.9836,78.31,20,0,9.66,1650148957,ML
316,316,marrakesh,31.6315,-8.0083,73.47,38,0,5.75,1650149008,MA
460,460,neuquen,-38.9516,-68.0591,69.69,43,0,6.91,1650149068,AR


### Hotel Map
* Create a dataframe named `hotel_df` that is the df created above only with the pertinent data.  See the prepared code below (`info_box_template`) to determine what data you will need for your final map.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels (aka "lodging") within 5000 meters.
* Utilize the Google Places API to find the first hotel search result.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [59]:
# Create dataframe and empty column
hotel_df = pd.DataFrame(narrow_df).reset_index(drop=True)
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country
0,11,mar del plata,-38.0023,-57.5575,65.08,89,0,3.00,1650148597,AR
1,51,buraydah,26.3260,43.9750,75.65,19,0,9.33,1650148901,SA
2,194,taoudenni,22.6783,-3.9836,78.31,20,0,9.66,1650148957,ML
3,316,marrakesh,31.6315,-8.0083,73.47,38,0,5.75,1650149008,MA
4,460,neuquen,-38.9516,-68.0591,69.69,43,0,6.91,1650149068,AR


In [36]:
# Loop through dataframe and add nearby hotel result
hotel_names = []

# Iterate through dataframe rows and update coordinates with latitude/longitude
for index, row in hotel_df.iterrows():
    target_coordinates = f"{row['Lat']}, {row['Lng']}"
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # Base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Run a request using our params dictionary
    response = requests.get(base_url, params=params)

    # JSON-ify response
    data = response.json()
    
    # Use try/except to add hotel names to a list
    try:
        hotel_names.append(data['results'][0]['name'])
    except:
        hotel_names.append(np.nan)
    

In [60]:
# View list of hotel names
hotel_names

['Gran Hotel Mar del Plata',
 'Mövenpick Qassim',
 nan,
 'Les Jardins de La Koutoubia',
 'Hotel del Comahue']

In [61]:
# Add hotel names into dataframe
hotel_df['Hotel Name'] = hotel_names

In [62]:
# Drop missing values 
hotel_df.dropna(inplace=True)

In [63]:
# NOTE: Do not change any of the code in this cell
# Analyze the code below for variables but these should be all columns in hotel_df

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
marker_locations = hotel_df[["Lat", "Lng"]]

In [65]:
# Redefine locations and weights for heat map
locations = df[["Lat", "Lng"]]
humidity = df["Humidity"]

fig = gmaps.figure()

# Create markers and add marker layer to map
markers = gmaps.marker_layer(marker_locations,info_box_content=hotel_info)
fig.add_layer(markers)

# Create heat layer and add heat layer to map
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100,
                                 point_radius=3)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))